## Comparing Sentinel C-Band SAR to Capella X-Band SAR (EDA Phase)

##### Step 1: Bring Sentinel Data in via the Google Earth Engine Python API and export it as a tif. The top of this repos README has instruction for how to pull data down from Capella's S3 bucket without an account.

In [ ]:
import json, numpy as np
from pyproj import Transformer
import ee, geemap, rasterio
import os

ee.Initialize(project='ee-krle4401')

# --- 1) Load Capella metadata ---
json_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/data/raw/scene_shanghai/CAPELLA_C11_SP_GEO_HH_20250320045730_20250320045802_extended.json"
with open(json_path, "r") as f:
    meta = json.load(f)

# Geotransform (UTM 51N, EPSG:32651)
gt = meta["collect"]["image"]["image_geometry"]["geotransform"]
rows = int(meta["collect"]["image"]["rows"])
cols = int(meta["collect"]["image"]["columns"])
x0, px_w, rot_x, y0, rot_y, px_h_neg = gt
px_h = abs(px_h_neg)

# Corners in UTM (no rotation for this product)
width_m  = px_w * cols
height_m = px_h * rows
corners_utm = np.array([
    [x0,             y0            ],  # TL
    [x0 + width_m,   y0            ],  # TR
    [x0 + width_m,   y0 - height_m ],  # BR
    [x0,             y0 - height_m ],  # BL
    [x0,             y0            ],  # close
], dtype=float)

# Reproject to lon/lat
transformer = Transformer.from_crs(32651, 4326, always_xy=True)
lons, lats = transformer.transform(corners_utm[:,0], corners_utm[:,1])
footprint_ll = [[float(lon), float(lat)] for lon, lat in zip(lons, lats)]
centroid_lon = float(np.mean(lons[:-1])); centroid_lat = float(np.mean(lats[:-1]))

# --- 2) Correct center time from JSON ---
# Prefer center_pixel.center_time; fall back to start/stop if needed
center_iso = (meta.get("collect", {})
                  .get("image", {})
                  .get("center_pixel", {})
                  .get("center_time"))
if center_iso is None:
    center_iso = meta["collect"]["start_timestamp"]  # safe fallback

capella_date = ee.Date(center_iso)
print("Capella center time:", center_iso)

# AOI polygon
aoi = ee.Geometry.Polygon([footprint_ll])

# --- 3) Find closest Sentinel scene (HH if available, else VV/VH) ---
window_days = 180
s1_hh = (ee.ImageCollection('COPERNICUS/S1_GRD')
         .filterBounds(aoi)
         .filterDate(capella_date.advance(-window_days,'day'),
                     capella_date.advance(window_days,'day'))
         .filter(ee.Filter.listContains('transmitterReceiverPolarisation','HH')))

def add_time_diff(img):
    return img.set('time_diff_days', img.date().difference(capella_date, 'day').abs())

s1_hh_sorted = s1_hh.map(add_time_diff).sort('time_diff_days')

if s1_hh_sorted.size().getInfo() > 0:
    s1_img  = ee.Image(s1_hh_sorted.first()); s1_band = 'HH'
    s1_vis  = {'bands':[s1_band], 'min':-25, 'max':0}
    print("Closest Sentinel‑1 HH:", s1_img.date().format().getInfo())
else:
    s1_iw = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(aoi)
             .filterDate(capella_date.advance(-window_days,'day'),
                         capella_date.advance(window_days,'day'))
             .filter(ee.Filter.eq('instrumentMode','IW'))
             .sort('system:time_start'))
    s1_img = ee.Image(s1_iw.first())
    bands  = s1_img.bandNames().getInfo()
    s1_band = 'VV' if 'VV' in bands else 'VH'
    s1_vis  = {'bands':[s1_band],
               'min': -22 if s1_band=='VV' else -27,
               'max':   0 if s1_band=='VV' else  -5}
    print("Closest Sentinel‑1 (IW):", s1_img.date().format().getInfo(), "| Band:", s1_band)

# --- 4) Map & local Capella overlay ---
Map = geemap.Map(center=[centroid_lat, centroid_lon], zoom=13)
Map.addLayer(s1_img.select(s1_band), s1_vis, f"S1 {s1_band} (closest)")
Map.addLayer(aoi, {'color':'red'}, "Capella footprint (exact)")

capella_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/shanghai_hh_db.tif"  # adjust if needed
with rasterio.open(capella_path) as src:
    arr = src.read(1, masked=True).compressed()
    p2, p98 = np.percentile(arr, [2, 98])

Map.add_raster(capella_path, layer_name="Capella HH (local dB)",
               vmin=float(p2), vmax=float(p98), opacity=0.65)

Map.split_map(left_layer=f"S1 {s1_band} (closest)", right_layer="Capella HH (local dB)")
project_dir = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs"
os.makedirs(project_dir, exist_ok=True)
out_html = os.path.join(project_dir, "sentinel_capella_compare.html")
Map.to_html(out_html, title="Capella vs Sentinel (aligned)", width="100%", height="820px")
print("Saved: sentinel_capella_compare.html")


#### Since 

In [20]:
out_tif = os.path.join(project_dir, "sentinel_capella_roi.tif")
geemap.ee_export_image(
    s1_img.select(s1_band).clip(aoi),
    filename=out_tif,
    scale=10,
    region=aoi
)


Generating URL ...
Please wait ...
Data downloaded to /Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/sentinel_capella_roi.tif


#### Convert Capella to a dB product

In [3]:
import json, numpy as np, rasterio, os
from rasterio.enums import Resampling

cap_dn_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/data/raw/scene_shanghai/CAPELLA_C11_SP_GEO_HH_20250320045730_20250320045802.tif"   # the float/int image with values in thousands
cap_json    = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/data/raw/scene_shanghai/CAPELLA_C11_SP_GEO_HH_20250320045730_20250320045802_extended.json"
cap_db_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/capella_sigma0_db_float32.tif"

# --- parse metadata for scaling ---
with open(cap_json, "r") as f:
    meta = json.load(f)

img_meta   = meta.get("collect", {}).get("image", {})
radiometry = (img_meta.get("radiometry") or "").lower()
scale_db   = img_meta.get("scale_factor_db", None) or img_meta.get("db_per_dn", None)
offset_db  = img_meta.get("offset_db", 0.0)
scale_lin  = img_meta.get("scale_factor", None) or img_meta.get("linear_scale", None)
pixel_val  = (img_meta.get("pixel_value") or img_meta.get("pixel_units") or "").lower()

def dn_to_db(arr):
    a = arr.astype("float32")
    # NoData as NaN
    a[a == 0] = np.nan

    # Case A: explicit dB-per-DN
    if isinstance(scale_db, (int, float)) and abs(scale_db) < 1.0:
        return a * float(scale_db) + float(offset_db)

    # Case B: linear scaling provided
    if isinstance(scale_lin, (int, float)) and scale_lin > 0:
        if "ampl" in pixel_val:
            return 20.0 * np.log10(a * float(scale_lin) + 1e-12) + float(offset_db)
        # assume POWER if not amplitude
        return 10.0 * np.log10(a * float(scale_lin) + 1e-12) + float(offset_db)

    # Case C: fallback heuristic (try POWER)
    return 10.0 * np.log10(a + 1e-12)

# --- stream-convert to float32 dB GeoTIFF ---
os.makedirs(os.path.dirname(cap_db_path), exist_ok=True)
with rasterio.open(cap_dn_path) as src:
    profile = src.profile.copy()
    profile.update(dtype="float32", count=1, nodata=np.nan, compress="deflate")
    with rasterio.open(cap_db_path, "w", **profile) as dst:
        block_h = src.block_shapes[0][0] if src.is_tiled else 1024
        block_w = src.block_shapes[0][1] if src.is_tiled else 1024
        for y in range(0, src.height, block_h):
            h = min(block_h, src.height - y)
            for x in range(0, src.width, block_w):
                w = min(block_w, src.width - x)
                win = rasterio.windows.Window(x, y, w, h)
                dn = src.read(1, window=win)
                db = dn_to_db(dn)
                dst.write(db.astype("float32"), 1, window=win)

print("Wrote Capella σ⁰(dB):", cap_db_path)


Wrote Capella σ⁰(dB): /Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/capella_sigma0_db_float32.tif


Even just looking between the two, the difference is staggering. It is worth reminding you that these are different bands. 

<p float="left">
  <img src="https://raw.githubusercontent.com/kllewers/Capella_Space_Dabbling/main/Sentinel_data.png" width="49%" />
  <img src="https://raw.githubusercontent.com/kllewers/Capella_Space_Dabbling/main/Sentinel_Capella_Data.png" width="49%" />
</p>


In [5]:
import numpy as np, rasterio, os
from rasterio.enums import Resampling
from rasterio.vrt import WarpedVRT
from rasterio.windows import Window

cap_db_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/capella_sigma0_db_float32.tif"    # from step 1
s1_db_path  = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/sentinel_capella_roi.tif"       # your clipped S1 (dB)
stack_path  = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/S1_CAP_overlap_stack.tif"

BLOCK = 1024
os.makedirs(os.path.dirname(stack_path), exist_ok=True)

with rasterio.open(cap_db_path) as cap_src, rasterio.open(s1_db_path) as s1_src:
    profile = cap_src.profile.copy()
    profile.update(count=2, dtype="float32", nodata=np.nan, compress="deflate", tiled=True,
                   blockxsize=BLOCK, blockysize=BLOCK)
    with rasterio.open(stack_path, "w", **profile) as dst:
        # Present S1 in Capella's grid
        with WarpedVRT(s1_src, crs=cap_src.crs, transform=cap_src.transform,
                       width=cap_src.width, height=cap_src.height,
                       resampling=Resampling.bilinear) as s1_vrt:
            for y in range(0, cap_src.height, BLOCK):
                for x in range(0, cap_src.width, BLOCK):
                    w = Window(x, y, min(BLOCK, cap_src.width-x), min(BLOCK, cap_src.height-y))
                    cap_tile = cap_src.read(1, window=w, out_dtype="float32")
                    s1_tile  = s1_vrt.read(1, window=w, out_dtype="float32")

                    # sanity masks (exclude nodata and silly values)
                    cap_tile[cap_tile <= -60] = np.nan
                    s1_tile[s1_tile <= -60]   = np.nan

                    both = np.isfinite(cap_tile) & np.isfinite(s1_tile)
                    cap_out = np.where(both, cap_tile, np.nan).astype("float32")
                    s1_out  = np.where(both, s1_tile,  np.nan).astype("float32")

                    dst.write(s1_out, 1, window=w)
                    dst.write(cap_out, 2, window=w)

print("Wrote overlap stack:", stack_path, "(band1=S1 dB, band2=Capella dB)")


Wrote overlap stack: /Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/S1_CAP_overlap_stack.tif (band1=S1 dB, band2=Capella dB)


In [ ]:
import numpy as np, rasterio
from rasterio.windows import Window

step = 20  # sample every 20 px each way
vals = []

with rasterio.open(stack_path) as src:
    H, W = src.height, src.width
    for y in range(0, H, step):
        h = min(step, H - y)
        for x in range(0, W, step):
            w = min(step, W - x)
            win = Window(x, y, w, h)
            s1 = src.read(1, window=win)
            ca = src.read(2, window=win)
            mask = np.isfinite(s1) & np.isfinite(ca)
            if mask.any():
                vals.append(np.column_stack([s1[mask], ca[mask]]))

arr = np.vstack(vals) if vals else np.empty((0,2), dtype="float32")
print("Sampled overlapping pixels:", arr.shape[0])
if arr.size:
    print("S1 mean (dB):", float(arr[:,0].mean()))
    print("Capella mean (dB):", float(arr[:,1].mean()))
    # optional: quick correlation
    corr = np.corrcoef(arr[:,0], arr[:,1])[0,1]
    print("Pearson r:", float(corr))
